# Pandas vs Vaex vs Polars

# The Tests:
1. Reading in the file. Speed Comparison
2. Compute metric of a column (mean, std)
3. Finding the unique count of a column
4. Comulative sum of a column
5. Group By Aggregation

# Vaex

In [2]:
import vaex as vx

In [3]:
%%time
df = vx.open("/content/test.parquet")  # Vaex uses the 'open' function to read data


CPU times: user 550 ms, sys: 89 ms, total: 639 ms
Wall time: 868 ms


In [4]:
%%time
df_mean = df["age"].mean(progress='widget')  # Vaex performs calculations lazily. The 'progress' argument ensures the calculation is performed immediately.


CPU times: user 236 ms, sys: 67.5 ms, total: 303 ms
Wall time: 283 ms


In [5]:
%%time
df_std = df["age"].std(progress='widget')


CPU times: user 1.09 s, sys: 129 ms, total: 1.22 s
Wall time: 969 ms


In [6]:
%%time
df_unique = df["prob"].unique(progress='widget')


CPU times: user 8.94 s, sys: 826 ms, total: 9.76 s
Wall time: 9.62 s


In [7]:
%%time
df_cumsum = df["age"].cumsum(progress='widget')


AttributeError: ignored

In [8]:
%%time
df_groupby = df.groupby(df.age, agg=vx.agg.mean(df.prob))  # Vaex uses 'agg' to specify the aggregation function in groupby

CPU times: user 486 ms, sys: 172 ms, total: 658 ms
Wall time: 519 ms


# Vaex Results
| Operation       | Time     |
|-----------------|----------|
| Read Parquet    | 868 ms   |
| Mean            | 283 ms   |
| STD             | 969 ms   |
| Unique          | 9.62 s   |
| Commulative Sum | ---      |
| Group By Mean   | 519 ms   |


# Pandas

In [23]:
import pandas as pd
import numpy as np

In [24]:
%%time
df = pd.read_parquet("test.parquet")

CPU times: user 706 ms, sys: 730 ms, total: 1.44 s
Wall time: 906 ms


In [25]:
%%time
df_mean = df["age"].mean()

CPU times: user 13.3 ms, sys: 0 ns, total: 13.3 ms
Wall time: 17 ms


In [26]:
%%time
df_std = df["age"].std()

CPU times: user 33.1 ms, sys: 38.4 ms, total: 71.5 ms
Wall time: 71.7 ms


In [27]:
%%time
df_unique = df["prob"].unique()

CPU times: user 1.88 s, sys: 141 ms, total: 2.02 s
Wall time: 2.03 s


In [28]:
%%time
df_cumsum = df["age"].cumsum()

CPU times: user 47.7 ms, sys: 24.1 ms, total: 71.8 ms
Wall time: 228 ms


In [29]:
%%time
df_groupby = df.groupby("age")["prob"].mean()

CPU times: user 149 ms, sys: 39.9 ms, total: 188 ms
Wall time: 198 ms


# Pandas Results
| Operation       | Time   |
|-----------------|--------|
| Read Parquet    | 906 ms |
| Mean            | 17 ms  |
| STD             | 71.7 ms|
| Unique          | 2.03 s |
| Cumulative Sum  | 228 ms |
| Group By Mean   | 198 ms |


In [16]:
import polars as pl

In [17]:
%%time
df = pl.read_parquet("test.parquet")

CPU times: user 1.25 s, sys: 237 ms, total: 1.48 s
Wall time: 1.48 s


In [18]:
%%time
df_mean = df["age"].mean()

CPU times: user 2.74 ms, sys: 2.67 ms, total: 5.41 ms
Wall time: 21.6 ms


In [19]:
%%time
df_std = df["age"].std()

CPU times: user 25.4 ms, sys: 121 ms, total: 146 ms
Wall time: 208 ms


In [20]:
%%time
df_unique = df["prob"].unique()

CPU times: user 1.59 s, sys: 113 ms, total: 1.7 s
Wall time: 1.86 s


In [21]:
%%time
df_cumsum = df["age"].cumsum()

CPU times: user 177 ms, sys: 314 ms, total: 491 ms
Wall time: 516 ms


In [22]:
%%time
df_groupby = df.groupby("age").agg(pl.col("prob").mean())

CPU times: user 308 ms, sys: 45.3 ms, total: 353 ms
Wall time: 213 ms


# Polars Results
| Operation       | Time    |
|-----------------|---------|
| Read Parquet    | 1.48 s  |
| Mean            | 21.6 ms |
| STD             | 208 ms  |
| Unique          | 1.86 s  |
| Cumulative Sum  | 516 ms  |
| Group By Mean   | 213 ms  |


## Performance Benchmark Summary

The table below summarizes the results of the performance benchmarking done on Vaex, Pandas, and Polars libraries:

| Operation         | Vaex     | Pandas  | Polars  |
|-------------------|---------:|--------:|--------:|
| Read Parquet      | 868 ms   | 906 ms  | 1.48 s  |
| Mean              | 283 ms   | 17 ms   | 21.6 ms |
| Standard Deviation| 969 ms   | 71.7 ms | 208 ms  |
| Unique            | 9.62 s   | 2.03 s  | 1.86 s  |
| Cumulative Sum    | N/A      | 228 ms  | 516 ms  |
| GroupBy Mean      | 519 ms   | 198 ms  | 213 ms  |

## Commentary

It is not easy to conclusively state which library is better as it depends on the specific use case, the nature of the data, the operations to be performed, and the size of the dataset. 

That said, here are some considerations:

1. **Vaex**: This shines when dealing with large datasets that cannot fit into memory as it uses a memory-efficient, lazy computation approach. In this benchmark, Vaex shows relatively longer execution time, especially for unique value counting and it doesn't natively support cumulative sum computation.

2. **Pandas**: It generally provides the fastest operation times for these benchmarks. It is a widely used, versatile library that is great for small-to-medium-sized datasets. However, for very large datasets, it may struggle with memory management.

3. **Polars**: This is a relative newcomer compared to Vaex and Pandas. It is slower than Pandas in these benchmarks, but it is designed to be fast and memory efficient for larger datasets, even outperforming Vaex in some of these benchmarks.

Given these results, if you are working with relatively smaller datasets (which can fit into memory), then Pandas would likely be the fastest and easiest to use. If you are working with larger datasets, both Vaex and Polars could be good options. Vaex has excellent memory efficiency, and Polars offers a balance of speed and memory efficiency.

It's worth noting that benchmarking results can vary based on factors such as hardware, dataset size and structure, and the specific operations being performed. It's often beneficial to run your own benchmarks based on your typical use cases to determine the best tool for your needs.
